In [ ]:
import glob
import pandas as pd
import numpy as np
import os
import csv
import sys
import random

year = '2022'

In [ ]:
#Read CSV file containing the people assigned an occupation (9 categories)
## CSV obtained after running spensr_notebook_5
df_persons_NE_occupation_dir = '/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only' # use your path
df_persons_NE_occupation_file = os.path.join(df_persons_NE_occupation_dir, "df_after_occupation.csv")
df_persons_NE_occupation = pd.read_csv(df_persons_NE_occupation_file, index_col=None, header=0)

In [ ]:
# Create a new column for the Occupation
df_persons_NE_occupation['Income'] = np.nan

## Start calculating the income per occupation type

### Occupation 2:

In [ ]:
## Income data

# Source: ONS
## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 3
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation2digitsocashetable3
## Year: 2021 revised edition of this dataset
## File downloaded: table32021revised.zip
## Table: Work Region Occupation SOC10 (2) Table 3.7a   Annual pay - Gross 2021.xls


# Select estimated min and max income values based on the percentile values given in previous source.
## These values are related to a specific region of England (in this case, West Midlands)
occupation_2_income = [18824,60531]

##########################################################################################

## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 15
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation4digitsoc2010ashetable15
## Year: 2021 revised edition of this dataset
## File downloaded: table152021revised.zip
## Table: Work Region Occupation SOC10 (4) Table 15.12  Gender pay gap 2021.xls

#### Gender pay gap per occupation type (MEAN value) WM values
occupation_2_gap = 0.108

##########################################################################################
##########################################################################################

## code developed:

# Name all occupation types to loop through them in the code:
occupations = [2]


for occupation in occupations:

    
    #Select the people of the df_occupation that is related to the selected OA area:
    (globals()[f"df_occupation_{occupation}"]) = df_persons_NE_occupation.loc[(df_persons_NE_occupation['Occupation'] == occupation)]
    
    # Select the Min and Max income values of each occupation category:
    occupation_income = (globals()[f"occupation_{occupation}_income"])
    
    
    # Calculate the income age slot
    ## this value will indicate the range in which agents will be assigned the income based on their age
    income_age_width_slot = (occupation_income[1] - occupation_income[0])/ 10
    
    
    ## Do the calculus just agent by agent
    for idx_person, person in (globals()[f"df_occupation_{occupation}"]).iterrows():
        
        # Identify the slot of income based on the age (UK values)
        ## An estimated range of income was guessed for each person based on their ages.
        ## The estimated income by age was identified from: ONS
        ### Earnings and hours worked, age group: ASHE Table 6
        ### Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/agegroupashetable6
        ## Year: 2021 revised edition of this dataset
        ## File downloaded: table62021revised.zip
        ### Table: Age Group Table 6.7a   Annual pay - Gross 2021.xls
        #### Agents aged 40-49 will earn more than any other, then agents aged 30-39, then 50-59, then 25-29, then 22-29, then 60+, then 18-21 and finally 16-17
                            
        
        # Define the income range based on the age
        if (person['Age'] <= 17):
            income_min = occupation_income[0] - (occupation_income[0]/1.1)
            income_max = occupation_income[0] - (occupation_income[0]/3)
        elif (person['Age'] > 17 and person['Age'] <= 21):
            income_min = occupation_income[0] - 2*income_age_width_slot
            income_max = occupation_income[0] + 0.15 * income_age_width_slot    
        elif (person['Age'] > 21 and person['Age'] < 30):
            income_min = occupation_income[0] - (income_age_width_slot/3)
            income_max = occupation_income[0] + 2 * income_age_width_slot
        elif (person['Age'] >= 30 and person['Age'] < 40):
            income_min = occupation_income[0] + 2.5 * income_age_width_slot
            income_max = occupation_income[0] + 7 * income_age_width_slot
        elif (person['Age'] >= 40 and person['Age'] < 50):
            income_min = occupation_income[0] + 5.5 * income_age_width_slot
            income_max = occupation_income[0] + 14 * income_age_width_slot 
        elif (person['Age'] >= 50 and person['Age'] < 60):
            income_min = occupation_income[0] + 2.5 * income_age_width_slot
            income_max = occupation_income[0] + 8 * income_age_width_slot
        elif (person['Age'] >= 60):
            income_min = occupation_income[0]
            income_max = occupation_income[0] + 4 * income_age_width_slot

            
        # Value identified as the income value of the person based on the occupation type and age range
        income_selected_by_occupation_age = random.randint(int(income_min), int(income_max))
        
     
        # Incorporate the gender gap
        ## If male, then increase the income half of the gender income gap
        ## If female, decrease the income half of the gender income gap
        if (person['Sex'] == 1):   # Male
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age + (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        else:
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age - (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        
                       
        # Transform the income value into an integer and round it to the 10 closest value
        income_chosen = int(income_selected_by_occupation_age_sex)
        income_round = round(income_chosen,-1)
        
        # Update the value in the column "Income"
        (globals()[f"df_occupation_{occupation}"]).at[idx_person,'Income'] = income_round
        

print('Job done. check the results')


In [ ]:
# Calculate min, max, mean and median values:

column = df_occupation_2["Income"]
min_value = column.min()
max_value = column.max()
mean_value = column.mean()
median_value = column.median()

print(min_value)
print(max_value)
print(mean_value)
print(median_value)

In [ ]:
# Differences from the mean
39030 - mean_value

In [ ]:
# Differences from the median
36755 - median_value

In [ ]:
# histogram of the income for occupation 2:

df_occupation_2.hist(column='Income', bins=50);

In [ ]:
# calculate the differences between observed (ONS) and calculated percentiles for occupation 2:

print(18824 - df_occupation_2.Income.quantile(0.1))
print(24962 - df_occupation_2.Income.quantile(0.2))
print(29521- df_occupation_2.Income.quantile(0.3))
print(32815 - df_occupation_2.Income.quantile(0.4))
print(40224 - df_occupation_2.Income.quantile(0.6))
print(44272 - df_occupation_2.Income.quantile(0.7))
print(50396 - df_occupation_2.Income.quantile(0.8))
print(60531 - df_occupation_2.Income.quantile(0.9))

##### Check the order of the median income per range of age

In [ ]:
df_income_16_17_occupation_2 = df_occupation_2.loc[(df_occupation_2['Age'] >=16) & (df_occupation_2['Age'] <=17)]

df_income_16_17_occupation_2["Income"].median()

In [ ]:
df_income_18_21_occupation_2 = df_occupation_2.loc[(df_occupation_2['Age'] >=18) & (df_occupation_2['Age'] <=21)]

df_income_18_21_occupation_2["Income"].median()

In [ ]:
df_income_22_29_occupation_2 = df_occupation_2.loc[(df_occupation_2['Age'] >=22) & (df_occupation_2['Age'] <=29)]

df_income_22_29_occupation_2["Income"].median()

In [ ]:
df_income_30_39_occupation_2 = df_occupation_2.loc[(df_occupation_2['Age'] >=30) & (df_occupation_2['Age'] <=39)]

df_income_30_39_occupation_2["Income"].median()

In [ ]:
df_income_40_49_occupation_2 = df_occupation_2.loc[(df_occupation_2['Age'] >=40) & (df_occupation_2['Age'] <=49)]

df_income_40_49_occupation_2["Income"].median()

In [ ]:
df_income_50_59_occupation_2 = df_occupation_2.loc[(df_occupation_2['Age'] >=50) & (df_occupation_2['Age'] <=59)]

df_income_50_59_occupation_2["Income"].median()

In [ ]:
df_income_60_120_occupation_2 = df_occupation_2.loc[(df_occupation_2['Age'] >=60)]

df_income_60_120_occupation_2["Income"].median()

In [ ]:
# Export data as csv file:
df_occupation_2_saved = df_occupation_2

df_occupation_2.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_occupation_2.csv', encoding='utf-8', header=True)


### Occupation 3

In [ ]:
## Income data

# Source: ONS
## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 3
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation2digitsocashetable3
## Year: 2021 revised edition of this dataset
## File downloaded: table32021revised.zip
## Table: Work Region Occupation SOC10 (2) Table 3.7a   Annual pay - Gross 2021.xls


# Select estimated min and max income values based on the percentile values given in previous source.
## These values are related to a specific region of England (in this case, WM)
occupation_3_income = [17043,52082]

##########################################################################################

## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 15
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation4digitsoc2010ashetable15
## Year: 2021 revised edition of this dataset
## File downloaded: table152021revised.zip
## Table: Work Region Occupation SOC10 (4) Table 15.12  Gender pay gap 2021.xls

#### Gender pay gap per occupation type (MEAN value) WM values
occupation_3_gap = 0.272

##########################################################################################
##########################################################################################

## code developed:

# Name all occupation types to loop through them in the code:
occupations = [3]


for occupation in occupations:

    
    #Select the people of the df_occupation that is related to the selected OA area:
    (globals()[f"df_occupation_{occupation}"]) = df_persons_NE_occupation.loc[(df_persons_NE_occupation['Occupation'] == occupation)]
    
    # Select the Min and Max income values of each occupation category:
    occupation_income = (globals()[f"occupation_{occupation}_income"])
    
    
    # Calculate the income age slot
    ## this value will indicate the range in which agents will be assigned the income based on their age
    income_age_width_slot = (occupation_income[1] - occupation_income[0])/ 10
    
    
    ## Do the calculus just agent by agent
    for idx_person, person in (globals()[f"df_occupation_{occupation}"]).iterrows():
        
        # Identify the slot of income based on the age (UK values)
        ## An estimated range of income was guessed for each person based on their ages.
        ## The estimated income by age was identified from: ONS
        ### Earnings and hours worked, age group: ASHE Table 6
        ### Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/agegroupashetable6
        ## Year: 2021 revised edition of this dataset
        ## File downloaded: table62021revised.zip
        ### Table: Age Group Table 6.7a   Annual pay - Gross 2021.xls
        #### Agents aged 40-49 will earn more than any other, then agents aged 30-39, then 50-59, then 25-29, then 22-29, then 60+, then 18-21 and finally 16-17
                    
        
        # Define the income range based on the age
        if (person['Age'] <= 17):
            income_min = occupation_income[0] - (occupation_income[0]/2)
            income_max = occupation_income[0] - (occupation_income[0]/3)
        elif (person['Age'] > 17 and person['Age'] <= 21):
            income_min = occupation_income[0] - income_age_width_slot
            income_max = occupation_income[0] + 0.2 * income_age_width_slot    
        elif (person['Age'] > 21 and person['Age'] < 30):
            income_min = occupation_income[0] - (income_age_width_slot/3)
            income_max = occupation_income[0] + 4.5 * income_age_width_slot
        elif (person['Age'] >= 30 and person['Age'] < 40):
            income_min = occupation_income[0] + 1.5 * income_age_width_slot
            income_max = occupation_income[0] + 6.75 * income_age_width_slot
        elif (person['Age'] >= 40 and person['Age'] < 50):
            income_min = occupation_income[0] + 6 * income_age_width_slot
            income_max = occupation_income[0] + 11 * income_age_width_slot 
        elif (person['Age'] >= 50 and person['Age'] < 60):
            income_min = occupation_income[0] + 2 * income_age_width_slot
            income_max = occupation_income[0] + 7.25 * income_age_width_slot
        elif (person['Age'] >= 60):
            income_min = occupation_income[0]
            income_max = occupation_income[0] + 3 * income_age_width_slot

            
        # Value identified as the income value of the person based on the occupation type and age range
        income_selected_by_occupation_age = random.randint(int(income_min), int(income_max))
        
     
        # Incorporate the gender gap
        ## If male, then increase the income half of the gender income gap
        ## If female, decrease the income half of the gender income gap
        if (person['Sex'] == 1):   # Male
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age + (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        else:
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age - (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        
                       
        # Transform the income value into an integer and round it to the 10 closest value
        income_chosen = int(income_selected_by_occupation_age_sex)
        income_round = round(income_chosen,-1)
        
        # Update the value in the column "Income"
        (globals()[f"df_occupation_{occupation}"]).at[idx_person,'Income'] = income_round
        

print('Job done. check the results')

In [ ]:
# Calculate min, max, mean and median values:

column = df_occupation_3["Income"]
min_value = column.min()
max_value = column.max()
mean_value = column.mean()
median_value = column.median()

print(min_value)
print(max_value)
print(mean_value)
print(median_value)

In [ ]:
# Differences to the mean value
39845 -  mean_value


In [ ]:
# Differences to the medeian value
31061 - median_value

In [ ]:
# Histogram:
df_occupation_3.hist(column='Income', bins=50);

In [ ]:
# calculate the differences between observed (ONS) and calculated percentiles for occupation 2:

print(17043 - df_occupation_3.Income.quantile(0.1))
print(21811 - df_occupation_3.Income.quantile(0.2))
print(25000- df_occupation_3.Income.quantile(0.3))
print(27964 - df_occupation_3.Income.quantile(0.4))
#print(27491 - df_occupation_3.Income.quantile(0.5))
print(34253 - df_occupation_3.Income.quantile(0.6))
print(38854 - df_occupation_3.Income.quantile(0.7))
print(44141 - df_occupation_3.Income.quantile(0.8))
print(52082 - df_occupation_3.Income.quantile(0.9))


##### Check the order of the median income per range of age

In [ ]:
df_income_16_17_occupation_3 = df_occupation_3.loc[(df_occupation_3['Age'] >=16) & (df_occupation_3['Age'] <=17)]

df_income_16_17_occupation_3["Income"].median()

In [ ]:
df_income_18_21_occupation_3 = df_occupation_3.loc[(df_occupation_3['Age'] >=18) & (df_occupation_3['Age'] <=21)]

df_income_18_21_occupation_3["Income"].median()

In [ ]:
df_income_22_29_occupation_3 = df_occupation_3.loc[(df_occupation_3['Age'] >=22) & (df_occupation_3['Age'] <=29)]

df_income_22_29_occupation_3["Income"].median()

In [ ]:
df_income_30_39_occupation_3 = df_occupation_3.loc[(df_occupation_3['Age'] >=30) & (df_occupation_3['Age'] <=39)]

df_income_30_39_occupation_3["Income"].median()

In [ ]:
df_income_40_49_occupation_3 = df_occupation_3.loc[(df_occupation_3['Age'] >=40) & (df_occupation_3['Age'] <=49)]

df_income_40_49_occupation_3["Income"].median()

In [ ]:
df_income_50_59_occupation_3 = df_occupation_3.loc[(df_occupation_3['Age'] >=50) & (df_occupation_3['Age'] <=59)]

df_income_50_59_occupation_3["Income"].median()

In [ ]:
df_income_60_120_occupation_3 = df_occupation_3.loc[(df_occupation_3['Age'] >=60)]

df_income_60_120_occupation_3["Income"].median()

In [ ]:
df_occupation_3_saved = df_occupation_3

df_occupation_3_saved.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_occupation_3.csv', encoding='utf-8', header=True)


### Occupation 5

In [ ]:
## Income data

# Source: ONS
## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 3
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation2digitsocashetable3
## Year: 2021 revised edition of this dataset
## File downloaded: table32021revised.zip
## Table: Work Region Occupation SOC10 (2) Table 3.7a   Annual pay - Gross 2021.xls


# Select estimated min and max income values based on the percentile values given in previous source.
## These values are related to a specific region of England (in this case, WM)
occupation_5_income = [13882,36106]

##########################################################################################

## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 15
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation4digitsoc2010ashetable15
## Year: 2021 revised edition of this dataset
## File downloaded: table152021revised.zip
## Table: Work Region Occupation SOC10 (4) Table 15.12  Gender pay gap 2021.xls

#### Gender pay gap per occupation type (MEAN value) WM values
occupation_5_gap = 0.206

##########################################################################################
##########################################################################################

## code developed:

# Name all occupation types to loop through them in the code:
occupations = [5]


for occupation in occupations:

    
    #Select the people of the df_occupation that is related to the selected OA area:
    (globals()[f"df_occupation_{occupation}"]) = df_persons_NE_occupation.loc[(df_persons_NE_occupation['Occupation'] == occupation)]
    
    # Select the Min and Max income values of each occupation category:
    occupation_income = (globals()[f"occupation_{occupation}_income"])
    
    
    # Calculate the income age slot
    ## this value will indicate the range in which agents will be assigned the income based on their age
    income_age_width_slot = (occupation_income[1] - occupation_income[0])/ 10
    
    
    ## Do the calculus just agent by agent
    for idx_person, person in (globals()[f"df_occupation_{occupation}"]).iterrows():
        
        # Identify the slot of income based on the age (UK values)
        ## An estimated range of income was guessed for each person based on their ages.
        ## The estimated income by age was identified from: ONS
        ### Earnings and hours worked, age group: ASHE Table 6
        ### Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/agegroupashetable6
        ## Year: 2021 revised edition of this dataset
        ## File downloaded: table62021revised.zip
        ### Table: Age Group Table 6.7a   Annual pay - Gross 2021.xls
        #### Agents aged 40-49 will earn more than any other, then agents aged 30-39, then 50-59, then 25-29, then 22-29, then 60+, then 18-21 and finally 16-17
                    
        
        # Define the income range based on the age
        if (person['Age'] <= 17):
            income_min = occupation_income[0] - (occupation_income[0]/1.1)
            income_max = occupation_income[0] - (occupation_income[0]/2)
        elif (person['Age'] > 17 and person['Age'] <= 21):
            income_min = occupation_income[0] - income_age_width_slot*3
            income_max = occupation_income[0] + 1 * income_age_width_slot  
        elif (person['Age'] > 21 and person['Age'] < 30):
            income_min = occupation_income[0] + 0.1 * income_age_width_slot
            income_max = occupation_income[0] + 7 * income_age_width_slot
        elif (person['Age'] >= 30 and person['Age'] < 40):
            income_min = occupation_income[0] + 2 * income_age_width_slot
            income_max = occupation_income[0] + 8 * income_age_width_slot
        elif (person['Age'] >= 40 and person['Age'] < 50):
            income_min = occupation_income[0] + 5 * income_age_width_slot
            income_max = occupation_income[0] + 16 * income_age_width_slot 
        elif (person['Age'] >= 50 and person['Age'] < 60):
            income_min = occupation_income[0] + 2 * income_age_width_slot
            income_max = occupation_income[0] + 10 * income_age_width_slot
        elif (person['Age'] >= 60):
            income_min = occupation_income[0] - (occupation_income[0]/2)
            income_max = occupation_income[0] + 4 * income_age_width_slot


            
            
        # Value identified as the income value of the person based on the occupation type and age range
        income_selected_by_occupation_age = random.randint(int(income_min), int(income_max))
        
     
        # Incorporate the gender gap
        ## If male, then increase the income half of the gender income gap
        ## If female, decrease the income half of the gender income gap
        if (person['Sex'] == 1):   # Male
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age + (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        else:
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age - (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        
                       
        # Transform the income value into an integer and round it to the 10 closest value
        income_chosen = int(income_selected_by_occupation_age_sex)
        income_round = round(income_chosen,-1)
        
        # Update the value in the column "Income"
        (globals()[f"df_occupation_{occupation}"]).at[idx_person,'Income'] = income_round
        

print('Job done. check the results')

In [ ]:
# Calculate the min, max, mean and median income values:
column = df_occupation_5["Income"]
min_value = column.min()
max_value = column.max()
mean_value = column.mean()
median_value = column.median()

print(min_value)
print(max_value)
print(mean_value)
print(median_value)

In [ ]:
# Difference to the mean
27998 -  mean_value


In [ ]:
# Difference to the median
27070 - median_value

In [ ]:
# histogram:

df_occupation_5.hist(column='Income', bins=50);

In [ ]:
# calculate the differences between observed (ONS) and calculated percentiles for occupation 2:

print(13882 - df_occupation_5.Income.quantile(0.1))
print(18342 - df_occupation_5.Income.quantile(0.2))
print(21507- df_occupation_5.Income.quantile(0.3))
print(24178 - df_occupation_5.Income.quantile(0.4))
#print( - df_occupation_5.Income.quantile(0.5))
print(29997 - df_occupation_5.Income.quantile(0.6))
print(32117 - df_occupation_5.Income.quantile(0.7))
print(36106 - df_occupation_5.Income.quantile(0.8))




##### Check the order of the median income per range of age

In [ ]:
df_income_16_17_occupation_5 = df_occupation_5.loc[(df_occupation_5['Age'] >=16) & (df_occupation_5['Age'] <=17)]

df_income_16_17_occupation_5["Income"].median()

In [ ]:
df_income_18_21_occupation_5 = df_occupation_5.loc[(df_occupation_5['Age'] >=18) & (df_occupation_5['Age'] <=21)]

df_income_18_21_occupation_5["Income"].median()

In [ ]:
df_income_22_29_occupation_5 = df_occupation_5.loc[(df_occupation_5['Age'] >=22) & (df_occupation_5['Age'] <=29)]

df_income_22_29_occupation_5["Income"].median()

In [ ]:
df_income_30_39_occupation_5 = df_occupation_5.loc[(df_occupation_5['Age'] >=30) & (df_occupation_5['Age'] <=39)]

df_income_30_39_occupation_5["Income"].median()

In [ ]:
df_income_40_49_occupation_5 = df_occupation_5.loc[(df_occupation_5['Age'] >=40) & (df_occupation_5['Age'] <=49)]

df_income_40_49_occupation_5["Income"].median()

In [ ]:
df_income_50_59_occupation_5 = df_occupation_5.loc[(df_occupation_5['Age'] >=50) & (df_occupation_5['Age'] <=59)]

df_income_50_59_occupation_5["Income"].median()

In [ ]:
df_income_60_120_occupation_5 = df_occupation_5.loc[(df_occupation_5['Age'] >=60)]

df_income_60_120_occupation_5["Income"].median()

In [ ]:
df_occupation_5_saved = df_occupation_5

df_occupation_5_saved.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_occupation_5.csv', encoding='utf-8', header=True)


### Occupation 7

In [ ]:
## Income data

# Source: ONS
## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 3
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation2digitsocashetable3
## Year: 2021 revised edition of this dataset
## File downloaded: table32021revised.zip
## Table: Work Region Occupation SOC10 (2) Table 3.7a   Annual pay - Gross 2021.xls


# Select estimated min and max income values based on the percentile values given in previous source.
## These values are related to a specific region of England (in this case, WM)
occupation_7_income = [5611,27647]

##########################################################################################

## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 15
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation4digitsoc2010ashetable15
## Year: 2021 revised edition of this dataset
## File downloaded: table152021revised.zip
## Table: Work Region Occupation SOC10 (4) Table 15.12  Gender pay gap 2021.xls

#### Gender pay gap per occupation type (MEAN value) WM values
occupation_7_gap = 0.033

##########################################################################################
##########################################################################################

## code developed:

# Name all occupation types to loop through them in the code:
occupations = [7]


for occupation in occupations:

    
    #Select the people of the df_occupation that is related to the selected OA area:
    (globals()[f"df_occupation_{occupation}"]) = df_persons_NE_occupation.loc[(df_persons_NE_occupation['Occupation'] == occupation)]
    
    # Select the Min and Max income values of each occupation category:
    occupation_income = (globals()[f"occupation_{occupation}_income"])
    
    
    # Calculate the income age slot
    ## this value will indicate the range in which agents will be assigned the income based on their age
    income_age_width_slot = (occupation_income[1] - occupation_income[0])/ 10
    
    
    ## Do the calculus just agent by agent
    for idx_person, person in (globals()[f"df_occupation_{occupation}"]).iterrows():
        
        # Identify the slot of income based on the age (UK values)
        ## An estimated range of income was guessed for each person based on their ages.
        ## The estimated income by age was identified from: ONS
        ### Earnings and hours worked, age group: ASHE Table 6
        ### Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/agegroupashetable6
        ## Year: 2021 revised edition of this dataset
        ## File downloaded: table62021revised.zip
        ### Table: Age Group Table 6.7a   Annual pay - Gross 2021.xls
        #### Agents aged 40-49 will earn more than any other, then agents aged 30-39, then 50-59, then 25-29, then 22-29, then 60+, then 18-21 and finally 16-17
                    
        
        # Define the income range based on the age
        if (person['Age'] <= 17):
            income_min = occupation_income[0] - (occupation_income[0]/1.1)
            income_max = occupation_income[0] - (occupation_income[0]/10)
        elif (person['Age'] > 17 and person['Age'] <= 21):
            income_min = occupation_income[0] - (occupation_income[0]/1.1)
            income_max = occupation_income[0] + 3.5 * income_age_width_slot  
        elif (person['Age'] > 21 and person['Age'] < 30):
            income_min = occupation_income[0] + 0.3 * income_age_width_slot
            income_max = occupation_income[0] + 8 * income_age_width_slot
        elif (person['Age'] >= 30 and person['Age'] < 40):
            income_min = occupation_income[0] + 1 * income_age_width_slot
            income_max = occupation_income[0] + 12 * income_age_width_slot
        elif (person['Age'] >= 40 and person['Age'] < 50):
            income_min = occupation_income[0] + 4 * income_age_width_slot
            income_max = occupation_income[0] + 15 * income_age_width_slot 
        elif (person['Age'] >= 50 and person['Age'] < 60):
            income_min = occupation_income[0] + 2 * income_age_width_slot
            income_max = occupation_income[0] + 12 * income_age_width_slot
        elif (person['Age'] >= 60):
            income_min = occupation_income[0] + 0.4 * income_age_width_slot
            income_max = occupation_income[0] + 6 * income_age_width_slot

            
        # Value identified as the income value of the person based on the occupation type and age range
        income_selected_by_occupation_age = random.randint(int(income_min), int(income_max))
        
     
        # Incorporate the gender gap
        ## If male, then increase the income half of the gender income gap
        ## If female, decrease the income half of the gender income gap
        if (person['Sex'] == 1):   # Male
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age + (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        else:
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age - (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        
                       
        # Transform the income value into an integer and round it to the 10 closest value
        income_chosen = int(income_selected_by_occupation_age_sex)
        income_round = round(income_chosen,-1)
        
        # Update the value in the column "Income"
        (globals()[f"df_occupation_{occupation}"]).at[idx_person,'Income'] = income_round
        

print('Job done. check the results')

In [ ]:
# Calculate the min, max, mean and median income value:
column = df_occupation_7["Income"]
min_value = column.min()
max_value = column.max()
mean_value = column.mean()
median_value = column.median()

print(min_value)
print(max_value)
print(mean_value)
print(median_value)

In [ ]:
# difference to the mean
15825 -  mean_value


In [ ]:
# difference to the median
14440 - median_value

In [ ]:
# histogram
df_occupation_7.hist(column='Income', bins=50);

In [ ]:
# calculate the differences between observed (ONS) and calculated percentiles for occupation 2:

print(5611 - df_occupation_7.Income.quantile(0.1))
print(7911 - df_occupation_7.Income.quantile(0.2))
print(9859 - df_occupation_7.Income.quantile(0.3))
print(12071 - df_occupation_7.Income.quantile(0.4))
#print( - df_occupation_5.Income.quantile(0.5))
print(17326 - df_occupation_7.Income.quantile(0.6))
print(19686 - df_occupation_7.Income.quantile(0.7))
print(22876 - df_occupation_7.Income.quantile(0.8))
print(27647 - df_occupation_7.Income.quantile(0.9))


##### Check the order of the median income per range of age

In [ ]:
df_income_16_17_occupation_7 = df_occupation_7.loc[(df_occupation_7['Age'] >=16) & (df_occupation_7['Age'] <=17)]

df_income_16_17_occupation_7["Income"].median()

In [ ]:
df_income_18_21_occupation_7 = df_occupation_7.loc[(df_occupation_7['Age'] >=18) & (df_occupation_7['Age'] <=21)]

df_income_18_21_occupation_7["Income"].median()

In [ ]:
df_income_22_29_occupation_7 = df_occupation_7.loc[(df_occupation_7['Age'] >=22) & (df_occupation_7['Age'] <=29)]

df_income_22_29_occupation_7["Income"].median()

In [ ]:
df_income_30_39_occupation_7 = df_occupation_7.loc[(df_occupation_7['Age'] >=30) & (df_occupation_7['Age'] <=39)]

df_income_30_39_occupation_7["Income"].median()

In [ ]:
df_income_40_49_occupation_7 = df_occupation_7.loc[(df_occupation_7['Age'] >=40) & (df_occupation_7['Age'] <=49)]

df_income_40_49_occupation_7["Income"].median()

In [ ]:
df_income_50_59_occupation_7 = df_occupation_7.loc[(df_occupation_7['Age'] >=50) & (df_occupation_7['Age'] <=59)]

df_income_50_59_occupation_7["Income"].median()

In [ ]:
df_income_60_120_occupation_7 = df_occupation_7.loc[(df_occupation_7['Age'] >=60)]

df_income_60_120_occupation_7["Income"].median()

In [ ]:
df_occupation_7_saved = df_occupation_7

df_occupation_7_saved.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_occupation_7.csv', encoding='utf-8', header=True)


### Occupation 8

In [ ]:
## Income data

# Source: ONS
## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 3
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation2digitsocashetable3
## Year: 2021 revised edition of this dataset
## File downloaded: table32021revised.zip
## Table: Work Region Occupation SOC10 (2) Table 3.7a   Annual pay - Gross 2021.xls


# Select estimated min and max income values based on the percentile values given in previous source.
## These values are related to a specific region of England (in this case, WM)
occupation_8_income = [12507,36378]

##########################################################################################

## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 15
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation4digitsoc2010ashetable15
## Year: 2021 revised edition of this dataset
## File downloaded: table152021revised.zip
## Table: Work Region Occupation SOC10 (4) Table 15.12  Gender pay gap 2021.xls

#### Gender pay gap per occupation type (MEAN value) WM values
occupation_8_gap = 0.150

##########################################################################################
##########################################################################################

## code developed:

# Name all occupation types to loop through them in the code:
occupations = [8]


for occupation in occupations:

    
    #Select the people of the df_occupation that is related to the selected OA area:
    (globals()[f"df_occupation_{occupation}"]) = df_persons_NE_occupation.loc[(df_persons_NE_occupation['Occupation'] == occupation)]
    
    # Select the Min and Max income values of each occupation category:
    occupation_income = (globals()[f"occupation_{occupation}_income"])
    
    
    # Calculate the income age slot
    ## this value will indicate the range in which agents will be assigned the income based on their age
    income_age_width_slot = (occupation_income[1] - occupation_income[0])/ 10
    
    
    ## Do the calculus just agent by agent
    for idx_person, person in (globals()[f"df_occupation_{occupation}"]).iterrows():
        
        # Identify the slot of income based on the age (UK values)
        ## An estimated range of income was guessed for each person based on their ages.
        ## The estimated income by age was identified from: ONS
        ### Earnings and hours worked, age group: ASHE Table 6
        ### Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/agegroupashetable6
        ## Year: 2021 revised edition of this dataset
        ## File downloaded: table62021revised.zip
        ### Table: Age Group Table 6.7a   Annual pay - Gross 2021.xls
        #### Agents aged 40-49 will earn more than any other, then agents aged 30-39, then 50-59, then 25-29, then 22-29, then 60+, then 18-21 and finally 16-17
                    
        
        # Define the income range based on the age
        if (person['Age'] <= 17):
            income_min = occupation_income[0] - (occupation_income[0]/1.1)
            income_max = occupation_income[0] - (occupation_income[0]/2)
        elif (person['Age'] > 17 and person['Age'] <= 21):
            income_min = occupation_income[0] - 4 * income_age_width_slot
            income_max = occupation_income[0] - 2 * income_age_width_slot  
        elif (person['Age'] > 21 and person['Age'] < 30):
            income_min = occupation_income[0] - 0.5 * income_age_width_slot
            income_max = occupation_income[0] + 5 * income_age_width_slot
        elif (person['Age'] >= 30 and person['Age'] < 40):
            income_min = occupation_income[0] + 1 * income_age_width_slot
            income_max = occupation_income[0] + 6.5 * income_age_width_slot
        elif (person['Age'] >= 40 and person['Age'] < 50):
            income_min = occupation_income[0] + 4 * income_age_width_slot
            income_max = occupation_income[0] + 12.5 * income_age_width_slot 
        elif (person['Age'] >= 50 and person['Age'] < 60):
            income_min = occupation_income[0] + 1.5 * income_age_width_slot
            income_max = occupation_income[0] + 9.5 * income_age_width_slot
        elif (person['Age'] >= 60):
            income_min = occupation_income[0] - 0.6 * income_age_width_slot
            income_max = occupation_income[0] + 3.3 * income_age_width_slot

            
        # Value identified as the income value of the person based on the occupation type and age range
        income_selected_by_occupation_age = random.randint(int(income_min), int(income_max))
        
     
        # Incorporate the gender gap
        ## If male, then increase the income half of the gender income gap
        ## If female, decrease the income half of the gender income gap
        if (person['Sex'] == 1):   # Male
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age + (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        else:
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age - (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        
                       
        # Transform the income value into an integer and round it to the 10 closest value
        income_chosen = int(income_selected_by_occupation_age_sex)
        income_round = round(income_chosen,-1)
        
        # Update the value in the column "Income"
        (globals()[f"df_occupation_{occupation}"]).at[idx_person,'Income'] = income_round
        

print('Job done. check the results')

In [ ]:
# Calculate the min, max, mean and median income values:
column = df_occupation_8["Income"]
min_value = column.min()
max_value = column.max()
mean_value = column.mean()
median_value = column.median()

print(min_value)
print(max_value)
print(mean_value)
print(median_value)

In [ ]:
# Difference to the mean
24859 -  mean_value


In [ ]:
# Difference to the median:
23725 - median_value

In [ ]:
# Histogram
df_occupation_8.hist(column='Income', bins=50);

In [ ]:
# calculate the differences between observed (ONS) and calculated percentiles for occupation 2:

print(12507 - df_occupation_8.Income.quantile(0.1))
print(17515 - df_occupation_8.Income.quantile(0.2))
print(19682 - df_occupation_8.Income.quantile(0.3))
print(21156 - df_occupation_8.Income.quantile(0.4))
#print( - df_occupation_8.Income.quantile(0.5))
print(25908 - df_occupation_8.Income.quantile(0.6))
print(28987 - df_occupation_8.Income.quantile(0.7))
print(32051 - df_occupation_8.Income.quantile(0.8))
print(36378 - df_occupation_8.Income.quantile(0.9))

##### Check the order of the median income per range of age

In [ ]:
df_income_16_17_occupation_8 = df_occupation_8.loc[(df_occupation_8['Age'] >=16) & (df_occupation_8['Age'] <=17)]

df_income_16_17_occupation_8["Income"].median()

In [ ]:
df_income_18_21_occupation_8 = df_occupation_8.loc[(df_occupation_8['Age'] >=18) & (df_occupation_8['Age'] <=21)]

df_income_18_21_occupation_8["Income"].median()

In [ ]:
df_income_22_29_occupation_8 = df_occupation_8.loc[(df_occupation_8['Age'] >=22) & (df_occupation_8['Age'] <=29)]

df_income_22_29_occupation_8["Income"].median()

In [ ]:
df_income_30_39_occupation_8 = df_occupation_8.loc[(df_occupation_8['Age'] >=30) & (df_occupation_8['Age'] <=39)]

df_income_30_39_occupation_8["Income"].median()

In [ ]:
df_income_40_49_occupation_8 = df_occupation_8.loc[(df_occupation_8['Age'] >=40) & (df_occupation_8['Age'] <=49)]

df_income_40_49_occupation_8["Income"].median()

In [ ]:
df_income_50_59_occupation_8 = df_occupation_8.loc[(df_occupation_8['Age'] >=50) & (df_occupation_8['Age'] <=59)]

df_income_50_59_occupation_8["Income"].median()

In [ ]:
df_income_60_120_occupation_8 = df_occupation_8.loc[(df_occupation_8['Age'] >=60)]

df_income_60_120_occupation_8["Income"].median()

In [ ]:
df_occupation_8_saved = df_occupation_8

df_occupation_8_saved.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_occupation_8.csv', encoding='utf-8', header=True)


### Occupation 9

In [ ]:
## Income data

# Source: ONS
## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 3
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation2digitsocashetable3
## Year: 2021 revised edition of this dataset
## File downloaded: table32021revised.zip
## Table: Work Region Occupation SOC10 (2) Table 3.7a   Annual pay - Gross 2021.xls


# Select estimated min and max income values based on the percentile values given in previous source.
## These values are related to a specific region of England (in this case, WM)
occupation_9_income = [4171,32568]

##########################################################################################

## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 15
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation4digitsoc2010ashetable15
## Year: 2021 revised edition of this dataset
## File downloaded: table152021revised.zip
## Table: Work Region Occupation SOC10 (4) Table 15.12  Gender pay gap 2021.xls

#### Gender pay gap per occupation type (MEAN value) WM values
occupation_9_gap = 0.180

##########################################################################################
##########################################################################################

## code developed:

# Name all occupation types to loop through them in the code:
occupations = [9]


for occupation in occupations:

    
    #Select the people of the df_occupation that is related to the selected OA area:
    (globals()[f"df_occupation_{occupation}"]) = df_persons_NE_occupation.loc[(df_persons_NE_occupation['Occupation'] == occupation)]
    
    # Select the Min and Max income values of each occupation category:
    occupation_income = (globals()[f"occupation_{occupation}_income"])
    
    
    # Calculate the income age slot
    ## this value will indicate the range in which agents will be assigned the income based on their age
    income_age_width_slot = (occupation_income[1] - occupation_income[0])/ 10
    
    
    ## Do the calculus just agent by agent
    for idx_person, person in (globals()[f"df_occupation_{occupation}"]).iterrows():
        
        # Identify the slot of income based on the age (UK values)
        ## An estimated range of income was guessed for each person based on their ages.
        ## The estimated income by age was identified from: ONS
        ### Earnings and hours worked, age group: ASHE Table 6
        ### Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/agegroupashetable6
        ## Year: 2021 revised edition of this dataset
        ## File downloaded: table62021revised.zip
        ### Table: Age Group Table 6.7a   Annual pay - Gross 2021.xls
        #### Agents aged 40-49 will earn more than any other, then agents aged 30-39, then 50-59, then 25-29, then 22-29, then 60+, then 18-21 and finally 16-17
                    
        
        # Define the income range based on the age
        if (person['Age'] <= 17):
            income_min = occupation_income[0] - (occupation_income[0]/2)
            income_max = occupation_income[0] - (occupation_income[0]/4)
        elif (person['Age'] > 17 and person['Age'] <= 21):
            income_min = occupation_income[0] - 1*income_age_width_slot
            income_max = occupation_income[0] + 2.5 * income_age_width_slot    
        elif (person['Age'] > 21 and person['Age'] < 30):
            income_min = occupation_income[0] - 1 * income_age_width_slot 
            income_max = occupation_income[0] + 3.5 * income_age_width_slot
        elif (person['Age'] >= 30 and person['Age'] < 40):
            income_min = occupation_income[0] + 4 * income_age_width_slot
            income_max = occupation_income[0] + 8.5 * income_age_width_slot
        elif (person['Age'] >= 40 and person['Age'] < 50):
            income_min = occupation_income[0] + 7.5 * income_age_width_slot
            income_max = occupation_income[0] + 13 * income_age_width_slot 
        elif (person['Age'] >= 50 and person['Age'] < 60):
            income_min = occupation_income[0] + 4 * income_age_width_slot
            income_max = occupation_income[0] + 8.5 * income_age_width_slot
        elif (person['Age'] >= 60):
            income_min = occupation_income[0] + 0.5 * income_age_width_slot
            income_max = occupation_income[0] + 4.5 * income_age_width_slot

            
        # Value identified as the income value of the person based on the occupation type and age range
        income_selected_by_occupation_age = random.randint(int(income_min), int(income_max))
        
     
        # Incorporate the gender gap
        ## If male, then increase the income half of the gender income gap
        ## If female, decrease the income half of the gender income gap
        if (person['Sex'] == 1):   # Male
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age + (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        else:
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age - (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        
                       
        # Transform the income value into an integer and round it to the 10 closest value
        income_chosen = int(income_selected_by_occupation_age_sex)
        income_round = round(income_chosen,-1)
        
        # Update the value in the column "Income"
        (globals()[f"df_occupation_{occupation}"]).at[idx_person,'Income'] = income_round
        

print('Job done. check the results')

In [ ]:
# Calculate the min, max, mean and median income values:
column = df_occupation_9["Income"]
min_value = column.min()
max_value = column.max()
mean_value = column.mean()
median_value = column.median()

print(min_value)
print(max_value)
print(mean_value)
print(median_value)

In [ ]:
# Difference to the mean:
17679 -  mean_value

In [ ]:
# difference to the median:
17775 - median_value

In [ ]:
# Histogram:
df_occupation_9.hist(column='Income', bins=50);

In [ ]:
# calculate the differences between observed (ONS) and calculated percentiles for occupation 2:

print(4171 - df_occupation_9.Income.quantile(0.1))
print(6862 - df_occupation_9.Income.quantile(0.2))
print(9877 - df_occupation_9.Income.quantile(0.3))
print(14353 - df_occupation_9.Income.quantile(0.4))
#print( - df_occupation_9.Income.quantile(0.5))
print(19876 - df_occupation_9.Income.quantile(0.6))
print(22587 - df_occupation_9.Income.quantile(0.7))
print(25665 - df_occupation_9.Income.quantile(0.8))
print(32568 - df_occupation_9.Income.quantile(0.9))


##### Check the order of the median income per range of age

In [ ]:
df_income_16_17_occupation_9 = df_occupation_9.loc[(df_occupation_9['Age'] >=16) & (df_occupation_9['Age'] <=17)]

df_income_16_17_occupation_9["Income"].median()

In [ ]:
df_income_18_21_occupation_9 = df_occupation_9.loc[(df_occupation_9['Age'] >=18) & (df_occupation_9['Age'] <=21)]

df_income_18_21_occupation_9["Income"].median()

In [ ]:
df_income_22_29_occupation_9 = df_occupation_9.loc[(df_occupation_9['Age'] >=22) & (df_occupation_9['Age'] <=29)]

df_income_22_29_occupation_9["Income"].median()

In [ ]:
df_income_30_39_occupation_9 = df_occupation_9.loc[(df_occupation_9['Age'] >=30) & (df_occupation_9['Age'] <=39)]

df_income_30_39_occupation_9["Income"].median()

In [ ]:
df_income_40_49_occupation_9 = df_occupation_9.loc[(df_occupation_9['Age'] >=40) & (df_occupation_9['Age'] <=49)]

df_income_40_49_occupation_9["Income"].median()

In [ ]:
df_income_50_59_occupation_9 = df_occupation_9.loc[(df_occupation_9['Age'] >=50) & (df_occupation_9['Age'] <=59)]

df_income_50_59_occupation_9["Income"].median()

In [ ]:
df_income_60_120_occupation_9 = df_occupation_9.loc[(df_occupation_9['Age'] >=60)]

df_income_60_120_occupation_9["Income"].median()

In [ ]:
df_occupation_9_saved = df_occupation_9

df_occupation_9_saved.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_occupation_9.csv', encoding='utf-8', header=True)


### Occupation 4

In [ ]:
## Income data

# Source: ONS
## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 3
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation2digitsocashetable3
## Year: 2021 revised edition of this dataset
## File downloaded: table32021revised.zip
## Table: Work Region Occupation SOC10 (2) Table 3.7a   Annual pay - Gross 2021.xls


# Select estimated min and max income values based on the percentile values given in previous source.
## These values are related to a specific region of England (in this case, WM)
occupation_4_income = [7705,32348]

##########################################################################################

## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 15
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation4digitsoc2010ashetable15
## Year: 2021 revised edition of this dataset
## File downloaded: table152021revised.zip
## Table: Work Region Occupation SOC10 (4) Table 15.12  Gender pay gap 2021.xls

#### Gender pay gap per occupation type (MEAN value) WM values
occupation_4_gap = 0.073 

##########################################################################################
##########################################################################################

## code developed:

# Name all occupation types to loop through them in the code:
occupations = [4]


for occupation in occupations:

    
    #Select the people of the df_occupation that is related to the selected OA area:
    (globals()[f"df_occupation_{occupation}"]) = df_persons_NE_occupation.loc[(df_persons_NE_occupation['Occupation'] == occupation)]
    
    # Select the Min and Max income values of each occupation category:
    occupation_income = (globals()[f"occupation_{occupation}_income"])
    
    
    # Calculate the income age slot
    ## this value will indicate the range in which agents will be assigned the income based on their age
    income_age_width_slot = (occupation_income[1] - occupation_income[0])/ 10
    
    
    ## Do the calculus just agent by agent
    for idx_person, person in (globals()[f"df_occupation_{occupation}"]).iterrows():
        
        # Identify the slot of income based on the age (UK values)
        ## An estimated range of income was guessed for each person based on their ages.
        ## The estimated income by age was identified from: ONS
        ### Earnings and hours worked, age group: ASHE Table 6
        ### Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/agegroupashetable6
        ## Year: 2021 revised edition of this dataset
        ## File downloaded: table62021revised.zip
        ### Table: Age Group Table 6.7a   Annual pay - Gross 2021.xls
        #### Agents aged 40-49 will earn more than any other, then agents aged 30-39, then 50-59, then 25-29, then 22-29, then 60+, then 18-21 and finally 16-17
                    
        
        # Define the income range based on the age
        if (person['Age'] <= 17):
            income_min = occupation_income[0] - (occupation_income[0]/2)
            income_max = occupation_income[0] - (occupation_income[0]/4)
        elif (person['Age'] > 17 and person['Age'] <= 21):
            income_min = occupation_income[0] -1* income_age_width_slot
            income_max = occupation_income[0] + income_age_width_slot    
        elif (person['Age'] > 21 and person['Age'] < 30):
            income_min = occupation_income[0] - 0.5*income_age_width_slot
            income_max = occupation_income[0] + 5 * income_age_width_slot
        elif (person['Age'] >= 30 and person['Age'] < 40):
            income_min = occupation_income[0] + 3 * income_age_width_slot
            income_max = occupation_income[0] + 10.5 * income_age_width_slot
        elif (person['Age'] >= 40 and person['Age'] < 50):
            income_min = occupation_income[0] + 3.5 * income_age_width_slot
            income_max = occupation_income[0] + 14* income_age_width_slot 
        elif (person['Age'] >= 50 and person['Age'] < 60):
            income_min = occupation_income[0] + 3* income_age_width_slot
            income_max = occupation_income[0] + 10.5 * income_age_width_slot
        elif (person['Age'] >= 60):
            income_min = occupation_income[0]
            income_max = occupation_income[0] + 4* income_age_width_slot


            
        # Value identified as the income value of the person based on the occupation type and age range
        income_selected_by_occupation_age = random.randint(int(income_min), int(income_max))
        
     
        # Incorporate the gender gap
        ## If male, then increase the income half of the gender income gap
        ## If female, decrease the income half of the gender income gap
        if (person['Sex'] == 1):   # Male
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age + (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        else:
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age - (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        
                       
        # Transform the income value into an integer and round it to the 10 closest value
        income_chosen = int(income_selected_by_occupation_age_sex)
        income_round = round(income_chosen,-1)
        
        # Update the value in the column "Income"
        (globals()[f"df_occupation_{occupation}"]).at[idx_person,'Income'] = income_round
        

print('Job done. check the results')

In [ ]:
# Calculate the min, max, mean and median income values:
column = df_occupation_4["Income"]
min_value = column.min()
max_value = column.max()
mean_value = column.mean()
median_value = column.median()

print(min_value)
print(max_value)
print(mean_value)
print(median_value)

In [ ]:
# Difference to the mean:
20311 -  mean_value

In [ ]:
# difference to the median
19782 - median_value

In [ ]:
# Histogram
df_occupation_4.hist(column='Income', bins=50);

In [ ]:
# calculate the differences between observed (ONS) and calculated percentiles for occupation 2:

print(7705 - df_occupation_4.Income.quantile(0.1))
print(11400 - df_occupation_4.Income.quantile(0.2))
print(14551 - df_occupation_4.Income.quantile(0.3))
print(17492 - df_occupation_4.Income.quantile(0.4))
#print( - df_occupation_9.Income.quantile(0.5))
print(21979 - df_occupation_4.Income.quantile(0.6))

print(24202 - df_occupation_4.Income.quantile(0.7))
print(27341 - df_occupation_4.Income.quantile(0.8))
print(32348 - df_occupation_4.Income.quantile(0.8))



##### Check the order of the median income per range of age

In [ ]:
df_income_16_17_occupation_4 = df_occupation_4.loc[(df_occupation_4['Age'] >=16) & (df_occupation_4['Age'] <=17)]

df_income_16_17_occupation_4["Income"].median()

In [ ]:
df_income_18_21_occupation_4 = df_occupation_4.loc[(df_occupation_4['Age'] >=18) & (df_occupation_4['Age'] <=21)]

df_income_18_21_occupation_4["Income"].median()

In [ ]:
df_income_22_29_occupation_4 = df_occupation_4.loc[(df_occupation_4['Age'] >=22) & (df_occupation_4['Age'] <=29)]

df_income_22_29_occupation_4["Income"].median()

In [ ]:
df_income_30_39_occupation_4 = df_occupation_4.loc[(df_occupation_4['Age'] >=30) & (df_occupation_4['Age'] <=39)]

df_income_30_39_occupation_4["Income"].median()

In [ ]:
df_income_40_49_occupation_4 = df_occupation_4.loc[(df_occupation_4['Age'] >=40) & (df_occupation_4['Age'] <=49)]

df_income_40_49_occupation_4["Income"].median()

In [ ]:
df_income_50_59_occupation_4 = df_occupation_4.loc[(df_occupation_4['Age'] >=50) & (df_occupation_4['Age'] <=59)]

df_income_50_59_occupation_4["Income"].median()

In [ ]:
df_income_60_120_occupation_4 = df_occupation_4.loc[(df_occupation_4['Age'] >=60)]

df_income_60_120_occupation_4["Income"].median()

In [ ]:
df_occupation_4_saved = df_occupation_4

df_occupation_4_saved.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_occupation_4.csv', encoding='utf-8', header=True)


### Occupation 1

In [ ]:
## Income data

# Source: ONS
## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 3
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation2digitsocashetable3
## Year: 2021 revised edition of this dataset
## File downloaded: table32021revised.zip
## Table: Work Region Occupation SOC10 (2) Table 3.7a   Annual pay - Gross 2021.xls


# Select estimated min and max income values based on the percentile values given in previous source.
## These values are related to a specific region of England (in this case, WM)
occupation_1_income = [16187, 60278]

##########################################################################################

## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 15
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation4digitsoc2010ashetable15
## Year: 2021 revised edition of this dataset
## File downloaded: table152021revised.zip
## Table: Work Region Occupation SOC10 (4) Table 15.12  Gender pay gap 2021.xls

#### Gender pay gap per occupation type (MEAN value) WM values
occupation_1_gap = 0.080

##########################################################################################
##########################################################################################

## code developed:

# Name all occupation types to loop through them in the code:
occupations = [1]


for occupation in occupations:

    
    #Select the people of the df_occupation that is related to the selected OA area:
    (globals()[f"df_occupation_{occupation}"]) = df_persons_NE_occupation.loc[(df_persons_NE_occupation['Occupation'] == occupation)]
    
    # Select the Min and Max income values of each occupation category:
    occupation_income = (globals()[f"occupation_{occupation}_income"])
    
    
    # Calculate the income age slot
    ## this value will indicate the range in which agents will be assigned the income based on their age
    income_age_width_slot = (occupation_income[1] - occupation_income[0])/ 10
    
    
    ## Do the calculus just agent by agent
    for idx_person, person in (globals()[f"df_occupation_{occupation}"]).iterrows():
        
        # Identify the slot of income based on the age (UK values)
        ## An estimated range of income was guessed for each person based on their ages.
        ## The estimated income by age was identified from: ONS
        ### Earnings and hours worked, age group: ASHE Table 6
        ### Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/agegroupashetable6
        ## Year: 2021 revised edition of this dataset
        ## File downloaded: table62021revised.zip
        ### Table: Age Group Table 6.7a   Annual pay - Gross 2021.xls
        #### Agents aged 40-49 will earn more than any other, then agents aged 30-39, then 50-59, then 25-29, then 22-29, then 60+, then 18-21 and finally 16-17
                    
        
        # Define the income range based on the age
        if (person['Age'] <= 17):
            income_min = occupation_income[0] - (occupation_income[0]/3)
            income_max = occupation_income[0] - (occupation_income[0]/5)
        elif (person['Age'] > 17 and person['Age'] <= 21):
            income_min = occupation_income[0] - (occupation_income[0]/3)
            income_max = occupation_income[0] + 1.5 * (occupation_income[0])   
        elif (person['Age'] > 21 and person['Age'] < 30):
            income_min = occupation_income[0] - (occupation_income[0]/5)
            income_max = occupation_income[0] + 2.25 * income_age_width_slot
        elif (person['Age'] >= 30 and person['Age'] < 40):
            income_min = occupation_income[0] + 1.5 * income_age_width_slot
            income_max = occupation_income[0] + 10 * income_age_width_slot
        elif (person['Age'] >= 40 and person['Age'] < 50):
            income_min = occupation_income[0] + 2.5 * income_age_width_slot
            income_max = occupation_income[0] + 20 * income_age_width_slot 
        elif (person['Age'] >= 50 and person['Age'] < 60):
            income_min = occupation_income[0] + 2 * income_age_width_slot
            income_max = occupation_income[0] + 12 * income_age_width_slot
        elif (person['Age'] >= 60):
            income_min = occupation_income[0]
            income_max = occupation_income[0] + 4 * income_age_width_slot


            
        # Value identified as the income value of the person based on the occupation type and age range
        income_selected_by_occupation_age = random.randint(int(income_min), int(income_max))
        
     
        # Incorporate the gender gap
        ## If male, then increase the income half of the gender income gap
        ## If female, decrease the income half of the gender income gap
        if (person['Sex'] == 1):   # Male
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age + (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        else:
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age - (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        
                       
        # Transform the income value into an integer and round it to the 10 closest value
        income_chosen = int(income_selected_by_occupation_age_sex)
        income_round = round(income_chosen,-1)
        
        # Update the value in the column "Income"
        (globals()[f"df_occupation_{occupation}"]).at[idx_person,'Income'] = income_round
        

print('Job done. check the results')

In [ ]:
# calculate the min, max, mean and median income values:
column = df_occupation_1["Income"]
min_value = column.min()
max_value = column.max()
mean_value = column.mean()
median_value = column.median()

print(min_value)
print(max_value)
print(mean_value)
print(median_value)

In [ ]:
# Difference to the mean:
45138 -  mean_value

In [ ]:
# Difference to the median:
38221 - median_value

In [ ]:
# Histogram:
df_occupation_1.hist(column='Income', bins=50);

In [ ]:
# calculate the differences between observed (ONS) and calculated percentiles for occupation 2:

print(23217 - df_occupation_1.Income.quantile(0.2))
print(28010 - df_occupation_1.Income.quantile(0.3))
print(33110 - df_occupation_1.Income.quantile(0.4))
#print( - df_occupation_9.Income.quantile(0.5))
print(43620 - df_occupation_1.Income.quantile(0.6))

print(50608 - df_occupation_1.Income.quantile(0.7))
print(60278 - df_occupation_1.Income.quantile(0.8))



##### Check the order of the median income per range of age

In [ ]:
df_income_16_17_occupation_1 = df_occupation_1.loc[(df_occupation_1['Age'] >=16) & (df_occupation_1['Age'] <=17)]

df_income_16_17_occupation_1["Income"].median()

In [ ]:
df_income_18_21_occupation_1 = df_occupation_1.loc[(df_occupation_1['Age'] >=18) & (df_occupation_1['Age'] <=21)]

df_income_18_21_occupation_1["Income"].median()

In [ ]:
df_income_22_29_occupation_1 = df_occupation_1.loc[(df_occupation_1['Age'] >=22) & (df_occupation_1['Age'] <=29)]

df_income_22_29_occupation_1["Income"].median()

In [ ]:
df_income_30_39_occupation_1 = df_occupation_1.loc[(df_occupation_1['Age'] >=30) & (df_occupation_1['Age'] <=39)]

df_income_30_39_occupation_1["Income"].median()

In [ ]:
df_income_40_49_occupation_1 = df_occupation_1.loc[(df_occupation_1['Age'] >=40) & (df_occupation_1['Age'] <=49)]

df_income_40_49_occupation_1["Income"].median()

In [ ]:
df_income_50_59_occupation_1 = df_occupation_1.loc[(df_occupation_1['Age'] >=50) & (df_occupation_1['Age'] <=59)]

df_income_50_59_occupation_1["Income"].median()

In [ ]:
df_income_60_120_occupation_1 = df_occupation_1.loc[(df_occupation_1['Age'] >=60)]

df_income_60_120_occupation_1["Income"].median()

In [ ]:
df_occupation_1_saved = df_occupation_1

df_occupation_1_saved.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_occupation_1.csv', encoding='utf-8', header=True)


### Occupation 6

In [ ]:
## Income data

# Source: ONS
## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 3
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation2digitsocashetable3
## Year: 2021 revised edition of this dataset
## File downloaded: table32021revised.zip
## Table: Work Region Occupation SOC10 (2) Table 3.7a   Annual pay - Gross 2021.xls


# Select estimated min and max income values based on the percentile values given in previous source.
## These values are related to a specific region of England (in this case, WM)
occupation_6_income = [6755,25026]

##########################################################################################

## Dataset: Earnings and hours worked, region by occupation by two-digit SOC: ASHE Table 3
## Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/regionbyoccupation2digitsocashetable3
## Year: 2021 revised edition of this dataset
## File downloaded: table32021revised.zip
## Table: Work Region Occupation SOC10 (2) Table 3.7a   Annual pay - Gross 2021.xls

#### Gender pay gap per occupation type (MEAN value) WM values
occupation_6_gap = 0.054

##########################################################################################
##########################################################################################

## code developed:

# Name all occupation types to loop through them in the code:
occupations = [6]


for occupation in occupations:

    
    #Select the people of the df_occupation that is related to the selected OA area:
    (globals()[f"df_occupation_{occupation}"]) = df_persons_NE_occupation.loc[(df_persons_NE_occupation['Occupation'] == occupation)]
    
    # Select the Min and Max income values of each occupation category:
    occupation_income = (globals()[f"occupation_{occupation}_income"])
    
    
    # Calculate the income age slot
    ## this value will indicate the range in which agents will be assigned the income based on their age
    income_age_width_slot = (occupation_income[1] - occupation_income[0])/ 10
    
    
    ## Do the calculus just agent by agent
    for idx_person, person in (globals()[f"df_occupation_{occupation}"]).iterrows():
        
        # Identify the slot of income based on the age (UK values)
        ## An estimated range of income was guessed for each person based on their ages.
        ## The estimated income by age was identified from: ONS
        ### Earnings and hours worked, age group: ASHE Table 6
        ### Link: https://www.ons.gov.uk/employmentandlabourmarket/peopleinwork/earningsandworkinghours/datasets/agegroupashetable6
        ## Year: 2021 revised edition of this dataset
        ## File downloaded: table62021revised.zip
        ### Table: Age Group Table 6.7a   Annual pay - Gross 2021.xls
        #### Agents aged 40-49 will earn more than any other, then agents aged 30-39, then 50-59, then 25-29, then 22-29, then 60+, then 18-21 and finally 16-17
                    
        
        # Define the income range based on the age
        if (person['Age'] <= 17):
            income_min = occupation_income[0] - (occupation_income[0]/4)
            income_max = occupation_income[0] - (occupation_income[0]/10)
        elif (person['Age'] > 17 and person['Age'] <= 21):
            income_min = occupation_income[0] - (occupation_income[0]/4)
            income_max = occupation_income[0] + 1.5 * (occupation_income[0])   
        elif (person['Age'] > 21 and person['Age'] < 30):
            income_min = occupation_income[0] 
            income_max = occupation_income[0] + 8 * income_age_width_slot
        elif (person['Age'] >= 30 and person['Age'] < 40):
            income_min = occupation_income[0] + 1 * income_age_width_slot
            income_max = occupation_income[0] + 10 * income_age_width_slot
        elif (person['Age'] >= 40 and person['Age'] < 50):
            income_min = occupation_income[0] + 2.5 * income_age_width_slot
            income_max = occupation_income[0] + 16 * income_age_width_slot 
        elif (person['Age'] >= 50 and person['Age'] < 60):
            income_min = occupation_income[0] + 2.5 * income_age_width_slot
            income_max = occupation_income[0] + 10 * income_age_width_slot
        elif (person['Age'] >= 60):
            income_min = occupation_income[0] + 1 * income_age_width_slot
            income_max = occupation_income[0] + 5 * income_age_width_slot


            
        # Value identified as the income value of the person based on the occupation type and age range
        income_selected_by_occupation_age = random.randint(int(income_min), int(income_max))
        
     
        # Incorporate the gender gap
        ## If male, then increase the income half of the gender income gap
        ## If female, decrease the income half of the gender income gap
        if (person['Sex'] == 1):   # Male
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age + (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        else:
            income_selected_by_occupation_age_sex = income_selected_by_occupation_age - (income_selected_by_occupation_age * (globals()[f"occupation_{occupation}_gap"])/2)
        
                       
        # Transform the income value into an integer and round it to the 10 closest value
        income_chosen = int(income_selected_by_occupation_age_sex)
        income_round = round(income_chosen,-1)
        
        # Update the value in the column "Income"
        (globals()[f"df_occupation_{occupation}"]).at[idx_person,'Income'] = income_round
        

print('Job done. check the results')

In [ ]:
# Calculate the min, max, mean and median income values:
column = df_occupation_6["Income"]
min_value = column.min()
max_value = column.max()
mean_value = column.mean()
median_value = column.median()

print(min_value)
print(max_value)
print(mean_value)
print(median_value)

In [ ]:
# difference to the mean:
15569 -  mean_value

In [ ]:
# Difference to the median:
16031 - median_value

In [ ]:
# Histogram:
df_occupation_6.hist(column='Income', bins=50);

In [ ]:
# calculate the differences between observed (ONS) and calculated percentiles for occupation 2:

print(11091 - df_occupation_6.Income.quantile(0.25))
print(12022 - df_occupation_6.Income.quantile(0.3))
print(13844 - df_occupation_6.Income.quantile(0.4))
#print( - df_occupation_6.Income.quantile(0.5))
print(17175 - df_occupation_6.Income.quantile(0.6))

print(19103 - df_occupation_6.Income.quantile(0.7))
print(21408 - df_occupation_6.Income.quantile(0.8))


##### Check the order of the median income per range of age

In [ ]:
df_income_16_17_occupation_6 = df_occupation_6.loc[(df_occupation_6['Age'] >=16) & (df_occupation_6['Age'] <=17)]

df_income_16_17_occupation_6["Income"].median()

In [ ]:
df_income_18_21_occupation_6 = df_occupation_6.loc[(df_occupation_6['Age'] >=18) & (df_occupation_6['Age'] <=21)]

df_income_18_21_occupation_6["Income"].median()

In [ ]:
df_income_22_29_occupation_6 = df_occupation_6.loc[(df_occupation_6['Age'] >=22) & (df_occupation_6['Age'] <=29)]

df_income_22_29_occupation_6["Income"].median()

In [ ]:
df_income_30_39_occupation_6 = df_occupation_6.loc[(df_occupation_6['Age'] >=30) & (df_occupation_6['Age'] <=39)]

df_income_30_39_occupation_6["Income"].median()

In [ ]:
df_income_40_49_occupation_6 = df_occupation_6.loc[(df_occupation_6['Age'] >=40) & (df_occupation_6['Age'] <=49)]

df_income_40_49_occupation_6["Income"].median()

In [ ]:
df_income_50_59_occupation_6 = df_occupation_6.loc[(df_occupation_6['Age'] >=50) & (df_occupation_6['Age'] <=59)]

df_income_50_59_occupation_6["Income"].median()

In [ ]:
df_income_60_120_occupation_6 = df_occupation_6.loc[(df_occupation_6['Age'] >=60)]

df_income_60_120_occupation_6["Income"].median()

In [ ]:
# Save data as csv

In [ ]:
df_occupation_6_saved = df_occupation_6

df_occupation_6_saved.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_occupation_6.csv', encoding='utf-8', header=True)


## Concatenate all results in a dataframe

In [ ]:
frames = [df_occupation_1, df_occupation_2, df_occupation_3, df_occupation_4, df_occupation_5, df_occupation_6, df_occupation_7, df_occupation_8, df_occupation_9]

df_income_all = pd.concat(frames)

## Save the dataframe with all income values as a csv

In [ ]:
df_income_all_saved = df_income_all

df_income_all_saved.to_csv('/home/manon/Documents/Leeds/RAIM/code/RAIM/data/westmidlands/synthetic_pop_SPENSER/'+year+'/NE_only/df_income_all_saved.csv', encoding='utf-8', header=True)


## Check global results

In [ ]:
# Calculate min, max, mean and median income values:
column = df_income_all["Income"]
min_value = column.min()
max_value = column.max()
mean_value = column.mean()
median_value = column.median()

print(min_value)
print(max_value)
print(mean_value)
print(median_value)

In [ ]:
#difference to median
22602 - median_value

In [ ]:
# Difference to mean
26339 - mean_value

In [ ]:
# calculate the differences between observed (ONS) and calculated percentiles for occupation 2:

print(7993 - df_income_all.Income.quantile(0.1))
print(12607 - df_income_all.Income.quantile(0.2))
print(16679 - df_income_all.Income.quantile(0.3))
print(19560 - df_income_all.Income.quantile(0.4))
print(22602 - df_income_all.Income.quantile(0.5))
print(26126 - df_income_all.Income.quantile(0.6))
print(30605 - df_income_all.Income.quantile(0.7))
print(36523 - df_income_all.Income.quantile(0.8))
print(45580 - df_income_all.Income.quantile(0.9))

In [ ]:
print(df_income_all.Income.quantile(0.1))
print(df_income_all.Income.quantile(0.2))
print(df_income_all.Income.quantile(0.3))
print(df_income_all.Income.quantile(0.4))
print(df_income_all.Income.quantile(0.5))
print(df_income_all.Income.quantile(0.6))
print(df_income_all.Income.quantile(0.7))
print(df_income_all.Income.quantile(0.8))
print(df_income_all.Income.quantile(0.9))

##### Check the order of the median income per range of age

In [ ]:
df_income_all_1 = df_income_all.loc[(df_income_all['Age'] >=16) & (df_income_all['Age'] <=17)]

df_income_all_1["Income"].median()

In [ ]:
df_income_all_2 = df_income_all.loc[(df_income_all['Age'] >=18) & (df_income_all['Age'] <=21)]

df_income_all_2["Income"].median()

In [ ]:
df_income_all_3 = df_income_all.loc[(df_income_all['Age'] >=22) & (df_income_all['Age'] <=29)]

df_income_all_3["Income"].median()

In [ ]:
df_income_all_4 = df_income_all.loc[(df_income_all['Age'] >=30) & (df_income_all['Age'] <=39)]

df_income_all_4["Income"].median()

In [ ]:
df_income_all_5 = df_income_all.loc[(df_income_all['Age'] >=40) & (df_income_all['Age'] <=49)]

df_income_all_5["Income"].median()

In [ ]:
df_income_all_6 = df_income_all.loc[(df_income_all['Age'] >=50) & (df_income_all['Age'] <=59)]

df_income_all_6["Income"].median()

In [ ]:
df_income_all_7 = df_income_all.loc[(df_income_all['Age'] >=60)]

df_income_all_7["Income"].median()